In [20]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit_aer import AerSimulator, Aer
from QArithmetic import add, mult, div, sub, cadd
from math import pi

In [35]:
a = QuantumRegister(6)
b = QuantumRegister(6)
c = QuantumRegister(3)
is_3 = QuantumRegister(1)
is_prime = QuantumRegister(1)
is_prime_c = ClassicalRegister(1)
ca = ClassicalRegister(6)
cb = ClassicalRegister(6)
cc = ClassicalRegister(3)
qc = QuantumCircuit(a,b,c,is_3, is_prime, ca,cb,cc, is_prime_c)

# init

# a = 3/4 = 000 100 / 000 011
qc.h(a[0])

qc.x(a[0])
qc.cx(a[0], a[2])
qc.x(a[0])

qc.cx(a[0], a[1])

# b = 2 # 010 000
qc.x(b[4])


# create the is_prime subroutine (in this simple case only look if the number is even or odd)
is_prime_qc = QuantumCircuit(a, b, c, is_prime, name="/ 2")

div(is_prime_qc, a, b, c, 3)

qc.append(is_prime_qc, list(range(0, 15)) + [qc.num_qubits-1])

# cnot
qc.cx(a[3], is_prime)

qc.append(is_prime_qc.inverse(), list(range(0, 15)) + [qc.num_qubits-1])

# +1 on non prime numbers
qc.x(is_prime)

# encode 1 in c
qc.x(c[0])

cadd(qc, is_prime, c, range(0, 3), 2)

qc.x(c[0])

# conditional is prime
is_prime_qc = QuantumCircuit(a, b, c, name="/ 2")

div(is_prime_qc, a, b, c, 3)

is_prime_gate = is_prime_qc.to_gate().control(1)

qc.append(is_prime_gate, [qc.num_qubits-1] + list(range(0, 15)))

# cnot
qc.cx(a[3], is_prime)

qc.append(is_prime_qc.inverse().to_gate().control(1), [qc.num_qubits-1] + list(range(0, 15)))


# qc.barrier()

qc.measure(a, (ca))
qc.measure(b, (cb))
qc.measure(c, (cc))
qc.measure(is_prime, (is_prime_c))

# Simulate the circuit.
simulator = AerSimulator()
qct = transpile(qc, simulator)
# print(qct)

result = Aer.get_backend('statevector_simulator').run(qct, shots=10).result()
counts = result.get_counts()
print(counts)
print(qc)


QiskitError: 'One or more instructions cannot be converted to a gate. "barrier" is not a gate instruction'

{'000 000000 000011': 512, '000 000000 000100': 512}
